# Templot Examples

We start by installing the package from Pypi, this is not necessary if the templot folder is present in the same diretory as this notebook.

In [ ]:
!pip install --user templot

## Importing dependencies

In [ ]:
%matplotlib inline

In [ ]:
import os
import pandas as pd
import matplotlib.animation as animation
import matplotlib.pyplot as plt 
from IPython.display import HTML #in order to visualize animations

## Data Downloading and preprocessing

We can use the `download_irep` function to download and parse the IREP dataset.

In [ ]:
from templot import download_irep
filepath = os.path.join('.', 'templot', 'data', 'df.csv')
if not os.path.exists(filepath):
    download_irep(filepath)

df = pd.read_csv(filepath)
df.head()

We can use the `add_regions` function that adds the corresponding Region, Department, or/and Commune by looking at the longitude and latitude columns.

In [ ]:
from templot import add_regions

df = add_regions(df, "LLX", "LLY", add=["regions", "departements"]) 

df.head()

Some of the functions require the dataset to be in a melted form; i.e. having one columns containing the values and another containing the corresponding year.

In [ ]:
df_melted = pd.melt(
    df,
    id_vars=df.columns & [
                'Identifiant',
                'Nom_Etablissement_x',
                'LLX',
                'LLY',
                'Regions',
                'Departements',
                'Communes'
            ],
    var_name='Annee',
    value_name='Quantite',
)
df_melted = df_melted[df_melted.Quantite != 0]
df_melted['Annee'] = df_melted['Annee'].apply(lambda x: int(x[-4:]))
df_melted.head()

##  Example 1: Plot Aggregated Map

Here we can create a map that shows the evolution of the average quantity in every region. The color refelects the average cummulative quantity over the entire period. 

In [ ]:
from templot import plot_aggregated_map
my_map = plot_aggregated_map(
    data=df,
    variables=[
        "Quantite2004",
        "Quantite2005",
        "Quantite2006",
        "Quantite2007",
        "Quantite2008",
        "Quantite2009",
    ],
    group="Regions",
    aggregation_method="average",
    height=300)

my_map
#If the map is not displayed (if you're using Edge or Chrome) uncomment the follwing lines
# from IPython.display import IFrame
# my_map.save("map.html")
# IFrame("map.html", width='100%', height=750)

We can also look at the evolution of the number of polluting comapnies in each department. The function will warn us if the number of departments is too high.

In [ ]:
my_map_dep = plot_aggregated_map(
    data=df,
    variables=[col for col in df.columns if "Quantite" in col], # All years
    group="Departements",
    aggregation_method="count",
    height=300)

my_map_dep
#If the map is not displayed (if you're using Edge or Chrome) uncomment the follwing lines
# from IPython.display import IFrame
# my_map_dep.save("map_dep.html")
# IFrame("map_dep.html", width='100%', height=750)

## Example 2: Plot Polar Bar Evolution 

Here we can create an animation that shows the evolution of the maximum quantities per region across all years.

In [ ]:
from templot import plot_polar_bar_evolution

anim = plot_polar_bar_evolution(
    df_melted,
    var="Quantite",
    year="Annee",
    agr="max",
    y_grid=False,
    x_grid=False,
    y_ticks=False,
)

HTML(anim.to_jshtml())

## Example 3: Plot Polar Bar Evolution Interactive
We can also look at an interactive version of that graph.

In [ ]:
from templot import plot_polar_bar_evolution_interactive

fig = plot_polar_bar_evolution_interactive(df_melted,
    var="Quantite",
    year="Annee",
    agr="max")

fig

## Example 4: Plot Pie Chart Interactive
Here we can look closely at two specific years and compare between them.

In [ ]:
from templot import plot_pie_chart_interactive


fig = plot_pie_chart_interactive(df, "Quantite", 2004, 2005, "Regions")

fig

## Example 5: Plot Top 10 Barchart
Finally we take a look at the top 10 worst offenders every year.

In [ ]:
from templot import plot_top10_barchart

# Delete a few outliers/incorrect values
df_melted.drop(df_melted.index[[80307, 78095, 73504]], inplace=True)

# Plot animation from 2003 to 2017 :
fig, ax = plt.subplots(figsize=(16, 9), dpi=220, facecolor='#F8F7F7')
ani = animation.FuncAnimation(
    fig,
    plot_top10_barchart,
    frames=range(2003, 2018),
    interval=1500,
    fargs=[
        df_melted,
        "Quantite",
        "Annee",
        "Regions",
        'Nom_Etablissement_x',
        'Les établissement émettant le plus de déchets dangereux de 2003 à 2017',
        'Déchets dangereux (t/an)',
    ],
)
HTML(ani.to_jshtml())